In [1]:
import pandas as pd
import numpy as np
from scipy.stats import truncnorm
from mesa import Agent, Model
from mesa.time import RandomActivation
from sklearn import ensemble

In [2]:
def get_truncated_normal(mean=0, sd=1, low=0, upp=10):
    return truncnorm(
        (low - mean) / sd, (upp - mean) / sd, loc=mean, scale=sd)

def generate_t_agents():
    agents = pd.DataFrame()
    ages = np.round(np.random.normal(36, 5, 10000), 0)
    males = np.zeros((9000,), dtype=int)
    females = np.ones((1000,), dtype=int)
    genders = np.concatenate((males, females), axis=None)
    reli = np.zeros((8000,), dtype=int)
    relo = np.ones((2000,), dtype=int)
    religions = np.concatenate((reli, relo), axis=None)
    np.random.shuffle(genders)
    np.random.shuffle(religions)
    X1 = get_truncated_normal(mean=.25, sd=.25, low=0, upp=1)
    X2 = get_truncated_normal(mean=.5, sd=.15, low=0, upp=1)
    X3 = get_truncated_normal(mean=.35, sd=.3, low=0, upp=1)
    X4 = get_truncated_normal(mean=.2, sd=.35, low=0, upp=1)
    X5 = get_truncated_normal(mean=.5, sd=.4, low=0, upp=1)
    X6 = get_truncated_normal(mean=.15, sd=.4, low=0, upp=1)
    agr_bhv = X1.rvs(10000)
    rel_fnt = X2.rvs(10000)
    rel_conv = X6.rvs(10000)
    hst_twd_for = X3.rvs(10000)
    lvl_rct_act = X4.rvs(10000)
    crt_agr_lvl = X5.rvs(10000)
    prob_threat = np.zeros((10000,), dtype=float)

    agents['ages'] = ages.astype(int)
    agents['gender'] = genders
    agents['religion'] = religions
    agents['agr_bhv'] = agr_bhv
    agents['rel_fnt'] = rel_fnt
    agents['rel_conv'] = rel_conv
    agents['hst_twd_for'] = hst_twd_for
    agents['lvl_rct_act'] = lvl_rct_act
    agents['crt_agr_lvl'] = crt_agr_lvl
    agents['prob_threat'] = prob_threat

    return agents

def generate_pred_agents():
    agents = pd.DataFrame()

    ages = np.round(np.random.normal(36, 5, 10000), 0)
    males = np.zeros((9000,), dtype=int)
    females = np.ones((1000,), dtype=int)
    genders = np.concatenate((males, females), axis=None)
    reli = np.zeros((8000,), dtype=int)
    relo = np.ones((2000,), dtype=int)
    religions = np.concatenate((reli, relo), axis=None)
    np.random.shuffle(genders)
    np.random.shuffle(religions)
    X1 = get_truncated_normal(mean=.25, sd=.25, low=0, upp=1)
    X2 = get_truncated_normal(mean=.5, sd=.15, low=0, upp=1)
    X3 = get_truncated_normal(mean=.35, sd=.3, low=0, upp=1)
    X4 = get_truncated_normal(mean=.2, sd=.35, low=0, upp=1)
    X5 = get_truncated_normal(mean=.5, sd=.4, low=0, upp=1)
    X6 = get_truncated_normal(mean=.15, sd=.4, low=0, upp=1)
    agr_bhv = X1.rvs(10000)
    rel_fnt = X2.rvs(10000)
    rel_conv = X6.rvs(10000)
    hst_twd_for = X3.rvs(10000)
    lvl_rct_act = X4.rvs(10000)
    crt_agr_lvl = X5.rvs(10000)
    prob_threat = X2.rvs(10000)

    agents['ages'] = ages.astype(int)
    agents['gender'] = genders
    agents['religion'] = religions
    agents['agr_bhv'] = agr_bhv
    agents['rel_fnt'] = rel_fnt
    agents['rel_conv'] = rel_conv
    agents['hst_twd_for'] = hst_twd_for
    agents['lvl_rct_act'] = lvl_rct_act
    agents['crt_agr_lvl'] = crt_agr_lvl
    agents['prob_threat'] = prob_threat

    return agents

In [3]:
def train_model(agents):
    rfg = ensemble.RandomForestRegressor()
    X = agents.drop(['prob_threat'], 1)
    Y = agents.prob_threat

    rfg.fit(X, Y)

    return rfg

In [32]:
class TerroristAgent(Agent):
    
    def __init__(self, unique_id, model, agent, pred_model):
        super().__init__(unique_id, model)
        self.pred_model = pred_model
        self.age = int(agent.ages)
        self.gender = int(agent.gender)
        self.religion = int(agent.religion)
        self.agr_bhv = float(agent.agr_bhv)
        self.rel_fnt = float(agent.rel_fnt)
        self.rel_conv = float(agent.rel_conv)
        self.hst_twd_for = float(agent.hst_twd_for)
        self.lvl_rct_act = float(agent.lvl_rct_act)
        self.crt_agr_lvl = float(agent.crt_agr_lvl)
        self.prob_threat = 0
        
    def step(self):
        if((self.agr_bhv >= .75) or (self.rel_fnt >= .75) or (self.hst_twd_for >= .75) or (self.crt_agr_lvl >= .65)):
            self.crt_agr_lvl += .005
        if((self.agr_bhv <= .25) or (self.rel_fnt >= .25) or (self.hst_twd_for <= .25) or (self.crt_agr_lvl <= .25)):
            self.crt_agr_lvl -= .005

        self.prob_threat = float(self.pred_model.predict([[self.age, self.gender, self.religion, self.agr_bhv, self.rel_fnt,
                                                self.rel_conv, self.hst_twd_for, self.lvl_rct_act, self.crt_agr_lvl]]))
        
        if(self.prob_threat >= .75):
            self.aggr_action()
        else:
            self.convert()

    def aggr_action(self):
        pass
    
    def convert(self):
        print('%d: %.5f' % (self.unique_id, self.prob_threat))

In [33]:
class MapModel(Model):
    
    pred_model = None
    
    def __init__(self):
        t_agents = generate_t_agents()
        pred_agents = generate_pred_agents()
        pred_model = train_model(pred_agents)
        self.schedule = RandomActivation(self)
        for x in range(len(t_agents)):
            a = TerroristAgent(x, self, t_agents[x:x+1], pred_model)
            self.schedule.add(a)
            
    def step(self):
        self.schedule.step()

In [34]:
empty_model = MapModel()

C:\Users\Blaine\AppData\Roaming\Python\Python37\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [35]:
empty_model.step()

505: 0.51225
9620: 0.49998
7748: 0.45640
4629: 0.42670
7320: 0.47103
6940: 0.47485
5580: 0.55720
2640: 0.53770
310: 0.48195
2762: 0.52851
2690: 0.52141
9202: 0.61030
3263: 0.51200
9979: 0.49802
6017: 0.39522
3447: 0.47142
744: 0.55459
7442: 0.39935
9438: 0.48093
2115: 0.46300
2312: 0.46193
4915: 0.44964
7282: 0.43511
6706: 0.53660
1020: 0.42444
4486: 0.50702
1690: 0.49038
5084: 0.52550
3290: 0.52012
9535: 0.49994
2124: 0.54575
3953: 0.49908
2276: 0.52707
6151: 0.45524
8164: 0.44745
375: 0.58888
3905: 0.44166
3394: 0.55233
7358: 0.47960
6509: 0.58586
7918: 0.57525
164: 0.59084
5569: 0.43771
6680: 0.49892
5604: 0.48310
4741: 0.54009
1818: 0.48726
494: 0.47632
8708: 0.48639
7029: 0.42109
7263: 0.54669
3675: 0.53567
9627: 0.55447
3158: 0.49366
5685: 0.53585
15: 0.48014
2319: 0.60661
928: 0.46662
4336: 0.42163
7664: 0.50754
9714: 0.46485
6369: 0.50334
5297: 0.40676
8624: 0.47912
642: 0.51829
4106: 0.56989
6417: 0.53863
7758: 0.62215
8024: 0.53633
5720: 0.41354
7236: 0.48890
3289: 0.47644
20

985: 0.48902
3538: 0.51547
8474: 0.48167
4253: 0.54971
3111: 0.57142
3167: 0.42696
4232: 0.52184
388: 0.53356
2527: 0.50117
4533: 0.55151
617: 0.48425
4754: 0.49504
5261: 0.39991
4205: 0.49162
2295: 0.58969
1246: 0.62071
6575: 0.59801
8095: 0.53189
9004: 0.48704
3812: 0.50311
8375: 0.56332
2605: 0.48134
6553: 0.51724
7962: 0.53855
1192: 0.54413
2219: 0.59752
1558: 0.45026
1776: 0.43912
7421: 0.57118
7856: 0.49070
2942: 0.60695
9605: 0.46651
7710: 0.58555
4616: 0.42288
3997: 0.48963
2564: 0.48151
8602: 0.51813
1646: 0.41006
4627: 0.55699
3205: 0.40518
3443: 0.45101
9434: 0.50429
3399: 0.43322
7823: 0.44567
6357: 0.51305
54: 0.27936
433: 0.42571
386: 0.44210
1797: 0.53300
4024: 0.51280
6053: 0.55561
6593: 0.45195
7424: 0.49246
5653: 0.48830
3877: 0.41626
3396: 0.52014
7765: 0.53813
1667: 0.51055
3499: 0.47314
1084: 0.52216
8679: 0.44455
2004: 0.50006
9068: 0.51391
8541: 0.46522
943: 0.50260
3014: 0.44830
7544: 0.50122
1198: 0.59043
9781: 0.59472
7859: 0.58750
4296: 0.52588
1422: 0.51624


6068: 0.49837
9150: 0.53841
7611: 0.56598
2781: 0.48453
3594: 0.51733
1215: 0.59251
3559: 0.42723
42: 0.53912
8442: 0.47880
7118: 0.45060
2973: 0.50713
3716: 0.49479
2170: 0.50085
3679: 0.46734
4796: 0.52355
7665: 0.41895
5565: 0.48548
6344: 0.45313
2049: 0.58340
8274: 0.54783
3665: 0.46204
9326: 0.42562
9467: 0.55052
5782: 0.47973
3929: 0.48793
8409: 0.57740
5143: 0.52334
1263: 0.46445
2432: 0.51022
3015: 0.44664
3016: 0.46604
1029: 0.44110
6341: 0.54154
9613: 0.48085
8004: 0.45159
4739: 0.45911
3596: 0.40719
5209: 0.48101
7180: 0.57358
4262: 0.54594
8995: 0.53165
6993: 0.49139
5153: 0.48505
1343: 0.51855
9065: 0.53432
274: 0.45038
480: 0.52066
3145: 0.39550
9983: 0.47750
3505: 0.46191
4538: 0.54164
1260: 0.53268
7926: 0.53375
7045: 0.56251
3743: 0.55784
1369: 0.64531
41: 0.47356
244: 0.44549
7770: 0.48565
4420: 0.55671
5704: 0.50700
7088: 0.46876
4561: 0.51339
5049: 0.53850
1693: 0.54115
6693: 0.52384
2394: 0.47419
8986: 0.53936
8616: 0.50873
8801: 0.57022
8131: 0.40287
9124: 0.52221

2578: 0.47153
8098: 0.42138
3556: 0.56762
7965: 0.44630
5980: 0.46796
5182: 0.54150
3054: 0.48764
7944: 0.47492
5506: 0.50696
8388: 0.47403
6329: 0.49978
2814: 0.49951
2595: 0.44376
5616: 0.60226
2562: 0.58741
161: 0.55353
7415: 0.56297
703: 0.59166
3259: 0.42593
4421: 0.45220
3163: 0.58588
1216: 0.48117
178: 0.49335
3310: 0.49054
2912: 0.56056
3414: 0.38550
5583: 0.48808
4076: 0.45692
4877: 0.48130
6094: 0.49126
7989: 0.59587
4552: 0.43282
5723: 0.42958
1497: 0.51274
5756: 0.50126
5696: 0.52918
7019: 0.42610
2382: 0.49796
5286: 0.49902
1340: 0.46170
8556: 0.50760
7805: 0.45863
1877: 0.42973
8122: 0.52659
6104: 0.43895
2252: 0.50831
3337: 0.51134
3970: 0.50405
2110: 0.43846
9530: 0.55930
2736: 0.45929
3843: 0.52722
7355: 0.44034
59: 0.55726
1668: 0.48298
2079: 0.47852
1509: 0.50520
5304: 0.45254
6208: 0.41562
2924: 0.59020
6821: 0.42922
2747: 0.45773
3218: 0.47985
2266: 0.41810
6207: 0.44040
9147: 0.44318
7669: 0.50505
7889: 0.52186
7957: 0.49193
3697: 0.54502
4458: 0.49800
2239: 0.550

9022: 0.59088
9924: 0.60516
6001: 0.42453
2876: 0.51548
8102: 0.44484
6805: 0.59354
7939: 0.60502
4020: 0.46003
4222: 0.49501
1471: 0.50741
2815: 0.48291
9796: 0.50257
8786: 0.52167
5651: 0.42320
6744: 0.43266
3920: 0.54007
6165: 0.46926
1223: 0.54162
4065: 0.40409
5812: 0.46513
6036: 0.62823
198: 0.46375
8119: 0.47649
436: 0.54905
9097: 0.48234
7445: 0.53756
1663: 0.39280
8828: 0.47664
9337: 0.34980
662: 0.50897
8821: 0.52094
593: 0.46692
3655: 0.53027
7010: 0.55377
3726: 0.47153
336: 0.51873
4677: 0.53635
5587: 0.43819
4432: 0.45119
5198: 0.51172
6838: 0.45927
5011: 0.50220
2464: 0.47182
573: 0.47608
7208: 0.45618
4390: 0.36585
656: 0.47780
3114: 0.49796
2817: 0.52882
7835: 0.43959
4917: 0.55961
2729: 0.57223
2296: 0.38510
8477: 0.52134
4442: 0.50560
7194: 0.55050
6177: 0.41616
7379: 0.46456
2797: 0.51727
9122: 0.59295
203: 0.54097
140: 0.47563
2098: 0.49829
4599: 0.47562
8316: 0.48303
5327: 0.51531
4305: 0.51745
5839: 0.50703
1410: 0.54747
7066: 0.55252
6512: 0.41867
5042: 0.42712
7

...

7933: 0.49617
4688: 0.53101
1890: 0.56711
5843: 0.52238
3727: 0.45660
7501: 0.59608
1459: 0.60353
361: 0.51663
7328: 0.52320
7248: 0.44825
7711: 0.42552
5534: 0.44018
313: 0.52921
8377: 0.55846
8527: 0.50464
3793: 0.57097
4865: 0.50245
8038: 0.55153
4410: 0.48597
8345: 0.49295
9063: 0.51344
8406: 0.54910
2727: 0.51503
2325: 0.42177
8833: 0.51157
2567: 0.52887
4916: 0.21642
7746: 0.43857
6883: 0.44241
8938: 0.44484
9964: 0.53879
4559: 0.43319
8614: 0.45356
7025: 0.44445
5844: 0.53047
9009: 0.53922
8783: 0.53705
899: 0.48807
5214: 0.48143
904: 0.46173
555: 0.48408
7999: 0.53958
1569: 0.63416
5471: 0.50790
1008: 0.47227
7901: 0.49278
547: 0.48114
2803: 0.48013
1941: 0.60153
9376: 0.49005
461: 0.53013
1400: 0.50026
7694: 0.52555
6338: 0.47479
354: 0.50732
5666: 0.42812
1504: 0.57364
9420: 0.48669
6018: 0.46451
2598: 0.40993
7788: 0.42614
6423: 0.51471
5784: 0.50560
7747: 0.43142
5324: 0.43873
8287: 0.44145
8662: 0.49587
4925: 0.52088
1825: 0.60102
9399: 0.60837
9850: 0.47294
9591: 0.48456


9942: 0.50481
7374: 0.30007
4864: 0.39492
2865: 0.43499
9092: 0.46986
5462: 0.54544
9861: 0.58616
2177: 0.51789
7568: 0.47845
3076: 0.51248
5826: 0.53364
9611: 0.45936
2157: 0.48250
6163: 0.49432
4104: 0.47092
6783: 0.46792
3300: 0.51053
7737: 0.50966
9151: 0.54799
1156: 0.49323
4922: 0.44527
9520: 0.51030
4142: 0.45338
2619: 0.47408
476: 0.54516
5016: 0.50565
6913: 0.56639
9336: 0.51862
8682: 0.48340
3902: 0.44405
5799: 0.40005
2823: 0.40131
154: 0.49423
5977: 0.52499
3436: 0.46637
5426: 0.58281
8908: 0.46905
798: 0.46479
225: 0.51363
711: 0.52399
6960: 0.44272
3385: 0.46901
715: 0.55177
7614: 0.39680
5111: 0.57402
8412: 0.54557
1839: 0.47592
3247: 0.49399
7401: 0.48093
4357: 0.57350
6758: 0.43848
1117: 0.56917
3858: 0.55668
3774: 0.58788
1936: 0.47140
6118: 0.48403
601: 0.43237
6562: 0.45992
8014: 0.50099
4832: 0.49047
6190: 0.43289
9484: 0.50165
7565: 0.48544
8006: 0.57244
8422: 0.48476
8397: 0.46088
360: 0.56630
4062: 0.56054
1334: 0.45896
5461: 0.51989
1110: 0.39902
3382: 0.56903


1512: 0.50313
5283: 0.52193
5748: 0.60920
6071: 0.50329
9709: 0.46034
9735: 0.47533
9300: 0.44156
4448: 0.45095
5145: 0.35804
5457: 0.51781
4671: 0.43934
1230: 0.52610
9655: 0.46750
2721: 0.48445
3557: 0.44781
2962: 0.48879
5326: 0.49140
5814: 0.49464
3680: 0.47871
2138: 0.54278
7489: 0.47138
383: 0.38873
7847: 0.42567
7087: 0.51065
9999: 0.50819
2665: 0.44159
364: 0.51853
3967: 0.38016
6803: 0.54650
3713: 0.40680
4230: 0.48390
9742: 0.54609
789: 0.48666
1923: 0.57002
5496: 0.54543
7887: 0.50870
1487: 0.46169
1957: 0.45936
9270: 0.47802
4611: 0.38479
6144: 0.41928
3189: 0.48564
1719: 0.47094
575: 0.54059
5463: 0.44952
4765: 0.50971
6939: 0.45708
7885: 0.55339
1053: 0.48781
9204: 0.51658
774: 0.42761
8754: 0.49561
4565: 0.52124
230: 0.49633
8338: 0.47584
9572: 0.45404
9951: 0.40661
209: 0.47696
9743: 0.45066
4247: 0.54458
5727: 0.42950
6402: 0.43353
4153: 0.36634
1734: 0.52358
5859: 0.60674
1685: 0.45701
32: 0.45132
6058: 0.50050
1140: 0.51819
2422: 0.56864
8419: 0.46464
349: 0.36411
20

6148: 0.48276
1466: 0.47360
8421: 0.56612
4070: 0.37987
3041: 0.50162
5384: 0.49585
2660: 0.41020
9016: 0.53907
432: 0.48205
423: 0.52381
4871: 0.50295
1763: 0.52618
4132: 0.48913
7441: 0.58323
7397: 0.56332
8552: 0.40411
3172: 0.49649
293: 0.55805
9902: 0.51247
7991: 0.54558
5406: 0.39053
551: 0.46037
5254: 0.60508
6904: 0.36894
473: 0.50604
1947: 0.47159
8047: 0.55403
1164: 0.51508
9390: 0.46078
3739: 0.42904
2861: 0.43899
728: 0.53468
9882: 0.40967
8868: 0.57449
8583: 0.44566
372: 0.51030
4057: 0.37807
5554: 0.50723
1737: 0.50725
4705: 0.49517
3925: 0.50256
5718: 0.56019
7738: 0.53664
3360: 0.46521
7895: 0.55051
6711: 0.51453
5874: 0.52551
390: 0.49684
6890: 0.46036
5322: 0.50776
2416: 0.46824
441: 0.58300
9600: 0.49479
4086: 0.52643
1050: 0.54271
2714: 0.38634
8276: 0.51683
546: 0.50667
8803: 0.53751
9880: 0.52451
4145: 0.48355
3350: 0.51502
2174: 0.50853
8304: 0.50820
8096: 0.45972
8851: 0.54696
1339: 0.48030
4081: 0.57544
6322: 0.66001
1116: 0.58188
338: 0.48247
4217: 0.46927
564

358: 0.48758
1950: 0.43610
8637: 0.53816
1015: 0.51089
9820: 0.54979
3282: 0.51214
3631: 0.61343
5608: 0.48969
7465: 0.50181
8604: 0.55857
9321: 0.53790
1779: 0.44872
1304: 0.51118
9915: 0.57805
4514: 0.41951
4593: 0.52725
4309: 0.55933
4666: 0.43289
1175: 0.46450
2329: 0.49818
4337: 0.57141
1375: 0.41588
9023: 0.38412
5668: 0.52921
2827: 0.48490
9548: 0.45554
8712: 0.52519
3512: 0.53699
5184: 0.42171
7929: 0.47669
1675: 0.41201
7598: 0.60597
5849: 0.47562
9113: 0.48327
3903: 0.47665
3837: 0.48997
7920: 0.56437
1774: 0.47848
3318: 0.54796
3438: 0.42002
6111: 0.45928
3122: 0.43666
5447: 0.43034
7909: 0.44906
8432: 0.53443
269: 0.49884
3209: 0.52996
5538: 0.46809
3994: 0.44619
3297: 0.46677
6425: 0.51950
2125: 0.43391
2793: 0.43739
3746: 0.46188
2596: 0.53767
813: 0.49235
6256: 0.47855
832: 0.56988
5449: 0.47743
3146: 0.46251
5069: 0.58564
224: 0.53311
8920: 0.49369
7247: 0.47350
586: 0.59351
9144: 0.57151
9236: 0.50845
6410: 0.53436
5370: 0.43556
8893: 0.46832
9118: 0.47530
5135: 0.4424

In [36]:
empty_model.step()

1211: 0.44039
5659: 0.51447
3519: 0.52030
2985: 0.53940
8577: 0.56538
7503: 0.58242
5410: 0.49369
1799: 0.44074
797: 0.51372
9874: 0.35607
382: 0.49013
9572: 0.45404
6825: 0.49279
5349: 0.46623
1056: 0.58808
8502: 0.60305
4917: 0.55961
5917: 0.51650
1102: 0.43590
4710: 0.41728
2524: 0.40307
7292: 0.46098
7824: 0.46367
249: 0.45852
1571: 0.55918
6801: 0.53458
4588: 0.48997
5371: 0.57009
2547: 0.50126
2216: 0.48290
1157: 0.48847
4924: 0.55998
6588: 0.51902
3817: 0.54378
5542: 0.42318
4124: 0.58784
2858: 0.47002
1156: 0.49323
4854: 0.53118
1231: 0.46019
1542: 0.48784
7516: 0.55842
4659: 0.54038
312: 0.45891
2355: 0.45093
1483: 0.54673
5247: 0.50839
9440: 0.51155
899: 0.48807
7680: 0.48111
9258: 0.38043
2729: 0.57223
599: 0.52202
8543: 0.49623
9251: 0.60024
9012: 0.56226
8838: 0.43917
8037: 0.44889
9077: 0.48131
950: 0.51867
7511: 0.46769
4026: 0.54168
3893: 0.47782
585: 0.49211
505: 0.51225
1444: 0.57683
7987: 0.54513
7984: 0.50737
2529: 0.45997
978: 0.52731
3768: 0.58563
8282: 0.44924
72

7608: 0.51476
5335: 0.46778
4643: 0.55646
1461: 0.47321
8778: 0.45934
9764: 0.55844
8761: 0.40546
4696: 0.52646
169: 0.43221
2679: 0.46164
2012: 0.47293
9748: 0.48966
5664: 0.46886
690: 0.53765
3257: 0.50374
8223: 0.48873
7859: 0.53900
964: 0.48221
8531: 0.61713
2551: 0.50813
8142: 0.47326
55: 0.46700
6253: 0.49911
9104: 0.48245
3931: 0.55038
2422: 0.56864
4251: 0.54781
4844: 0.47558
8839: 0.52006
5870: 0.55970
7610: 0.46541
693: 0.54806
2503: 0.38484
1662: 0.47516
1728: 0.51104
6876: 0.47956
7106: 0.47726
1322: 0.63778
7067: 0.50891
1714: 0.51009
5773: 0.51811
1736: 0.61644
3903: 0.47665
9453: 0.48871
597: 0.49547
2316: 0.49473
7705: 0.55802
2873: 0.45208
9422: 0.42204
2123: 0.55360
5560: 0.50342
9246: 0.52719
5397: 0.67920
8295: 0.42236
3167: 0.42696
7252: 0.42699
4649: 0.55281
1356: 0.54363
400: 0.47251
4736: 0.52590
4975: 0.40757
7895: 0.55051
7593: 0.48426
5613: 0.49595
8673: 0.51527
3934: 0.50945
4293: 0.50062
5452: 0.46308
9301: 0.55855
3605: 0.59386
7355: 0.44034
3611: 0.50938


9261: 0.51973
9460: 0.52196
6429: 0.66164
6638: 0.42803
6959: 0.52508
9806: 0.45592
602: 0.50634
4876: 0.52732
9533: 0.57047
2161: 0.49400
6329: 0.49978
3716: 0.49479
3180: 0.56058
6021: 0.40475
1793: 0.49670
1281: 0.53234
2014: 0.46198
6600: 0.41087
2052: 0.41860
3390: 0.48152
4647: 0.47920
9101: 0.50551
5465: 0.54826
8615: 0.41018
3410: 0.48500
2379: 0.53224
5824: 0.60814
7001: 0.55728
1786: 0.46882
4390: 0.36585
7092: 0.48546
5524: 0.53610
692: 0.45166
1375: 0.41588
1479: 0.47999
2120: 0.48106
5924: 0.45469
4093: 0.39863
8156: 0.50981
8913: 0.55348
15: 0.47726
422: 0.54330
8107: 0.45660
8786: 0.52167
2872: 0.47398
318: 0.49078
9496: 0.52245
4572: 0.44679
9676: 0.47850
2074: 0.42620
232: 0.52523
4727: 0.40185
8960: 0.43846
5384: 0.49585
6758: 0.43848
8572: 0.47148
7257: 0.54363
1268: 0.48975
734: 0.43836
8004: 0.45159
77: 0.50914
3940: 0.42826
784: 0.46751
799: 0.51760
6143: 0.51006
9274: 0.50288
7448: 0.56348
7177: 0.53509
2205: 0.54434
8322: 0.55490
8180: 0.42349
7174: 0.45378
4735

9180: 0.60727
4468: 0.46249
6842: 0.49907
4162: 0.50555
2493: 0.42517
2280: 0.54570
31: 0.51932
6583: 0.52681
3442: 0.49876
4828: 0.58482
5332: 0.45369
4503: 0.57467
7949: 0.53260
3783: 0.57734
1041: 0.51006
9416: 0.45804
6571: 0.57144
9866: 0.54422
848: 0.45138
3400: 0.47337
7772: 0.54342
5350: 0.54061
9118: 0.47530
7071: 0.54341
5054: 0.38645
1643: 0.53734
9556: 0.59747
9374: 0.63101
5994: 0.52911
5295: 0.50015
911: 0.43334
9944: 0.43747
3944: 0.50623
7412: 0.53564
2694: 0.47965
6400: 0.45547
8441: 0.48130
5851: 0.52726
21: 0.48293
7108: 0.44223
2118: 0.47657
6687: 0.47957
874: 0.48677
4926: 0.40949
8844: 0.51332
7737: 0.50966
1978: 0.54726
3405: 0.59738
2707: 0.50046
4167: 0.54183
8128: 0.44630
5188: 0.52194
9083: 0.47197
5711: 0.48776
4339: 0.46438
6044: 0.54056
6739: 0.38933
2783: 0.43108
3444: 0.48024
9640: 0.37138
4342: 0.52687
2731: 0.48949
741: 0.50747
3650: 0.53121
7418: 0.46166
8801: 0.57022
7098: 0.59537
4408: 0.51930
8599: 0.50957
1694: 0.50929
4456: 0.45681
6663: 0.51266


8139: 0.53829
6119: 0.51253
3769: 0.63084
6009: 0.53144
8051: 0.53771
8508: 0.45427
481: 0.54151
2056: 0.47190
7248: 0.44825
2568: 0.51758
6952: 0.52153
7299: 0.61054
8583: 0.45514
9948: 0.51204
6984: 0.39884
6875: 0.46336
2939: 0.47930
6453: 0.45302
854: 0.46303
2395: 0.47698
3848: 0.49494
9055: 0.47517
1813: 0.53141
7560: 0.52016
9527: 0.58908
9009: 0.53922
3900: 0.49578
5854: 0.44722
6513: 0.54192
7172: 0.48221
6011: 0.45055
9677: 0.50774
8244: 0.46154
9120: 0.55535
4411: 0.42627
828: 0.53804
7986: 0.50710
886: 0.47839
7602: 0.46822
7690: 0.46718
3179: 0.51686
2142: 0.53942
6636: 0.41565
1459: 0.60353
5299: 0.40380
9943: 0.52571
4267: 0.45308
8311: 0.55840
8798: 0.48224
8456: 0.48177
7385: 0.38329
2880: 0.57678
6116: 0.43532
2780: 0.49026
3860: 0.45977
8196: 0.55394
1173: 0.46068
3967: 0.38016
2766: 0.47409
8178: 0.46573
4576: 0.37945
7980: 0.41070
3306: 0.51208
33: 0.52583
9072: 0.48803
3677: 0.49617
6601: 0.51241
6582: 0.57657
9557: 0.46490
4555: 0.48451
8942: 0.45974
8911: 0.4806

...

6744: 0.43266
9782: 0.47964
1703: 0.50002
3035: 0.45798
7352: 0.49562
3294: 0.52418
4254: 0.58389
4537: 0.47202
2207: 0.51818
1134: 0.53587
4827: 0.59318
9960: 0.43012
8387: 0.51378
4237: 0.60011
7579: 0.56498
3258: 0.50556
4867: 0.44165
1486: 0.56145
1711: 0.55444
5068: 0.45739
1342: 0.49678
6881: 0.49585
2338: 0.56606
8022: 0.52350
1100: 0.51764
9796: 0.50361
9902: 0.51247
1821: 0.60227
5221: 0.44918
820: 0.51445
2307: 0.45999
7770: 0.48565
8791: 0.55737
1592: 0.44960
7868: 0.56607
1669: 0.65052
1168: 0.53958
9019: 0.45841
8984: 0.50283
915: 0.50226
8473: 0.56531
7749: 0.45547
7694: 0.52555
9376: 0.49005
7069: 0.46635
278: 0.51961
7245: 0.43047
2392: 0.50363
6241: 0.58196
2751: 0.39977
4741: 0.54009
5927: 0.43392
5307: 0.49463
8064: 0.47239
1129: 0.56328
178: 0.49335
1884: 0.49044
3548: 0.47105
5088: 0.50886
8203: 0.64349
1163: 0.50309
5186: 0.52235
1013: 0.48474
7797: 0.48879
8010: 0.55701
9711: 0.43514
4974: 0.53834
1254: 0.43823
2228: 0.57627
2274: 0.46740
4133: 0.48818
6037: 0.47

9672: 0.48529
1212: 0.37809
5510: 0.50222
1583: 0.54488
3344: 0.62949
7752: 0.47092
9694: 0.46089
134: 0.53354
6271: 0.36238
6359: 0.46992
6401: 0.47213
3132: 0.49299
3790: 0.63604
1547: 0.45897
4623: 0.57463
3543: 0.44610
5816: 0.49924
7553: 0.41651
8503: 0.48726
5447: 0.43034
9328: 0.56305
5555: 0.44441
64: 0.60719
4196: 0.57405
7406: 0.47219
3578: 0.49005
5558: 0.44831
359: 0.46570
8099: 0.47489
9559: 0.53948
385: 0.53368
7167: 0.44705
8846: 0.55347
1657: 0.50716
8802: 0.52297
6375: 0.50668
3772: 0.48120
2899: 0.56150
1497: 0.52390
2497: 0.49243
3647: 0.54105
9775: 0.42269
6765: 0.48407
7780: 0.58357
6524: 0.51475
5331: 0.44295
4016: 0.47992
4078: 0.47523
439: 0.43595
4359: 0.54228
8238: 0.51202
3564: 0.45512
1632: 0.49635
2299: 0.45765
9067: 0.52321
1611: 0.44056
5092: 0.54884
2601: 0.54402
6946: 0.51523
6806: 0.42304
4108: 0.45041
871: 0.44230
4231: 0.42186
27: 0.39295
6327: 0.44390
2833: 0.55390
8532: 0.53104
8809: 0.49777
9004: 0.48704
9585: 0.54396
9875: 0.49831
4018: 0.55309
8

5388: 0.52069
5222: 0.47744
2373: 0.52711
6014: 0.47479
1521: 0.48985
3756: 0.48690
625: 0.52855
2204: 0.56844
6999: 0.45086
6291: 0.57076
4983: 0.52949
1413: 0.50341
3733: 0.45196
5901: 0.55418
2665: 0.44159
4064: 0.48831
4440: 0.48673
7282: 0.45148
8394: 0.56614
6477: 0.47320
2733: 0.45692
4863: 0.51009
3462: 0.51098
7035: 0.52430
4930: 0.57464
9985: 0.39347
3794: 0.57611
5158: 0.57909
2891: 0.45214
3114: 0.49796
3951: 0.38081
131: 0.59600
9139: 0.56798
4981: 0.50862
6814: 0.56791
4486: 0.50702
610: 0.51460
7952: 0.51358
2788: 0.53074
1397: 0.45499
8746: 0.48464
2197: 0.41942
9705: 0.44037
1790: 0.45682
3699: 0.53742
9789: 0.53284
4821: 0.54401
5048: 0.52976
2083: 0.49593
6667: 0.48936
3864: 0.48310
5494: 0.48270
4946: 0.60415
6439: 0.55324
5248: 0.52856
2969: 0.56369
5190: 0.52930
1525: 0.46578
2848: 0.53410
9550: 0.47365
1574: 0.44696
6982: 0.53263
478: 0.46639
7304: 0.47000
3027: 0.50187
1037: 0.48373
1492: 0.55450
5913: 0.54481
486: 0.54362
2531: 0.35269
2708: 0.46529
9896: 0.466

6222: 0.55628
2314: 0.49574
8375: 0.56332
8234: 0.54844
7178: 0.56952
3242: 0.48348
7329: 0.52649
3084: 0.48913
2634: 0.50795
6568: 0.44860
8611: 0.46644
8403: 0.47503
5273: 0.51796
4931: 0.57710
638: 0.62609
8536: 0.56577
3036: 0.53173
8706: 0.56462
9760: 0.57272
7799: 0.45656
6674: 0.44445
8145: 0.44964
123: 0.55332
4513: 0.59638
7111: 0.55211
5045: 0.58153
6491: 0.52318
9068: 0.51391
7862: 0.56701
8702: 0.45940
3175: 0.49639
4327: 0.57699
4360: 0.45526
5151: 0.42841
808: 0.51543
669: 0.47629
3645: 0.47214
5961: 0.48714
700: 0.49464
2755: 0.44086
8636: 0.43123
8419: 0.46464
6869: 0.51122
2710: 0.51622
9671: 0.51794
1261: 0.43614
937: 0.53947
2641: 0.60145
9128: 0.37776
6056: 0.52274
7223: 0.43568
2010: 0.51566
7670: 0.51414
7369: 0.51697
1021: 0.44948
4285: 0.52401
6979: 0.54698
7468: 0.44699
3956: 0.44400
508: 0.49726
1744: 0.52785
6063: 0.62847
3488: 0.57397
4416: 0.37551
6850: 0.45228
7020: 0.53778
2162: 0.47976
704: 0.55622
8450: 0.43632
5268: 0.45881
8509: 0.54923
9204: 0.51658


4951: 0.45545
4624: 0.55189
8675: 0.43226
6425: 0.51950
6564: 0.49992
3849: 0.49454
5798: 0.57348
9497: 0.51315
5655: 0.48023
9041: 0.49147
9414: 0.52758
9036: 0.49405
7924: 0.42938
3886: 0.52961
6736: 0.50685
9513: 0.56203
8269: 0.55530
9419: 0.53538
5610: 0.44435
3541: 0.49772
9515: 0.40106
4904: 0.45638
6428: 0.45362
2675: 0.54515
6644: 0.43455
3717: 0.54647
3525: 0.35740
2064: 0.52878
9280: 0.45519
3724: 0.50043
9880: 0.52451
5993: 0.44457
7143: 0.56406
281: 0.43104
2631: 0.54062
1599: 0.54897
4822: 0.44336
1044: 0.52364
4784: 0.49134
3632: 0.44230
9624: 0.50591
8454: 0.48066
1216: 0.48807
6421: 0.42208
2946: 0.52664
1225: 0.53712
7537: 0.44340
7398: 0.42318
4129: 0.44910
9312: 0.54498
9916: 0.53611
7491: 0.49887
5492: 0.58427
6705: 0.53717
1029: 0.44110
2203: 0.57955
4044: 0.47561
8711: 0.48958
4198: 0.43698
3286: 0.59011
9834: 0.52701
2407: 0.50352
927: 0.56917
2330: 0.51395
404: 0.61516
3492: 0.50879
9500: 0.51515
8225: 0.49254
708: 0.60061
2862: 0.50958
9133: 0.51584
7432: 0.47